In [1]:
import plotly.express as px
import pandas as pd

from mainnet_launch.constants import *
from mainnet_launch.database.schema.full import *
from mainnet_launch.database.schema.postgres_operations import *
from mainnet_launch.database.schema.views import *


def get_readable_plan_table() -> pd.DataFrame:
    df = merge_tables_as_df(
        selectors=[
            TableSelector(RebalancePlans),
            TableSelector(
                Autopools,
                select_fields=[Autopools.symbol],
                join_on=Autopools.autopool_vault_address == RebalancePlans.autopool_vault_address,
            ),
        ]
    )
    return df


event_df = get_readable_rebalance_events_by_autopool(SILO_USD)
df = get_readable_plan_table()
df

,file_name,datetime_generated,autopool_vault_address,chain_id,solver_address,rebalance_type,destination_out,token_out,destination_in,token_in,...,in_dest_adj_apr,apr_delta,swap_offset_period,num_candidate_destinations,candidate_destinations_rank,projected_swap_cost,projected_net_gain,projected_gross_gain,projected_slippage,symbol
0,rebalance_plan_1753671656_0x0A2b94F6871c1D7A32...,2025-07-28 03:00:56+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0x9b163e15121816BE53F8d5c85fbefD6e6D9bebCd,0x6b31a94029fd7840d780191B6D63Fa0D269bd883,...,0.117924,0.117924,26.0,5.0,0.0,0.005530,0.266996,0.272525,0.016309,autoETH
1,rebalance_plan_1751856345_0x0A2b94F6871c1D7A32...,2025-07-07 02:45:45+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,autoETH
2,rebalance_plan_1736548221_0x0A2b94F6871c1D7A32...,2025-01-10 22:30:21+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,0x88794C65550DeB6b4087B7552eCf295113794410,...,0.089252,0.089252,46.0,5.0,0.0,0.078233,1.903875,1.982107,0.042596,autoETH
3,rebalance_plan_1742585421_0x0A2b94F6871c1D7A32...,2025-03-21 19:30:21+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,autoETH
4,rebalance_plan_1731483021_0x0A2b94F6871c1D7A32...,2024-11-13 07:30:21+00:00,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D,FromIdle,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,0x88794C65550DeB6b4087B7552eCf295113794410,...,0.070416,0.070416,48.0,7.0,0.0,0.009908,0.406662,0.416570,0.021364,autoETH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13287,rebalance_plan_1756110156_0x52F0D57Fb5D4780a37...,2025-08-25 08:22:36+00:00,0x52F0D57Fb5D4780a37164f918746f9BD51c684a3,1,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,siloETH
13288,rebalance_plan_1756088556_0x52F0D57Fb5D4780a37...,2025-08-25 02:22:36+00:00,0x52F0D57Fb5D4780a37164f918746f9BD51c684a3,1,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,siloETH
13289,rebalance_plan_1756101155_0x52F0D57Fb5D4780a37...,2025-08-25 05:52:35+00:00,0x52F0D57Fb5D4780a37164f918746f9BD51c684a3,1,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,siloETH
13290,rebalance_plan_1756119155_0x52F0D57Fb5D4780a37...,2025-08-25 10:52:35+00:00,0x52F0D57Fb5D4780a37164f918746f9BD51c684a3,1,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,siloETH


In [11]:
this_destinations = list(set([*event_df["destination_out"].values, *event_df["destination_in"].values]))

In [13]:
destinations_df = get_full_table_as_df(
    DestinationStates, where_clause=DestinationStates.destination_vault_address.in_(this_destinations)
)
destinations_df

,destination_vault_address,block,chain_id,incentive_apr,fee_apr,base_apr,points_apr,fee_plus_base_apr,total_apr_in,total_apr_out,underlying_token_total_supply,safe_total_supply,lp_token_spot_price,lp_token_safe_price,from_rebalance_plan,rebalance_plan_timestamp,rebalance_plan_key


In [17]:
autopool_destinations = get_full_table_as_df(
    AutopoolDestinationStates
)  # , where_clause=AutopoolDestinationStates.destination_vault_address.in_(this_destinations))
autopool_destinations

,destination_vault_address,autopool_vault_address,block,chain_id,owned_shares
0,0x903F21CBC0cFd49817dB219Cd216325fD4132034,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,21712129,1,0.000000
1,0xfB6f99FdF12E37Bfe3c4Cf81067faB10c465fb24,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,22184669,1,0.000000
2,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,21511571,1,162.727288
3,0x25cb41919d6B88e0D48108A4F5fe8FBb3744aFE1,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,22342307,1,0.000000
4,0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,22506180,1,373.662845
...,...,...,...,...,...
142925,0x2fE75AEd0E1EFA62e47CE356814D16D9eb6450d1,0xCb119265AA1195ea363D7A243aD56c73EA42Eb59,44428026,146,0.000000
142926,0x3D35b1E0Fe434848490427CE4DE13b6839365Bd2,0xCb119265AA1195ea363D7A243aD56c73EA42Eb59,44428026,146,0.000000
142927,0x4827788F01D7533710cbfa7C0b5991E8b99E277F,0xCb119265AA1195ea363D7A243aD56c73EA42Eb59,44439597,146,421858.307028
142928,0x2Af19422D9f81A4c71306a409A540fC63d7f623B,0xCb119265AA1195ea363D7A243aD56c73EA42Eb59,44451155,146,0.000000
